# インポート

In [1]:
import create_population
import pandas as pd
from pathlib import Path
import create_population
import preprocessing
from feature_producing import FeatureCreator
from training import Trainer
%load_ext autoreload


In [43]:
%autoreload

# 学習母集団の作成
期間を指定すると、学習母集団である（日付、race_id、horse_id）の組みを作る

In [66]:
population = create_population.create(
    from_ ="2023-01-01",
    to_ = "2023-12-31",
)

In [ ]:
# 各年の数を確認
population["date"].dt.year.value_counts().sort_index()

date
2018    48604
2019    47574
2020    48282
2021    47821
2022    47220
2023    47672
Name: count, dtype: int64

# データ加工

In [2]:
import preprocessing
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
%autoreload

In [74]:
#レース結果テーブルの前処理
results_preprocessd = preprocessing.process_results()

In [13]:
# 重複チェック
results_preprocessd.duplicated(subset=["race_id", "horse_id"]).sum()

np.int64(0)

In [76]:
# 欠損チェック
results_preprocessd.isnull().sum()

race_id        0
horse_id       0
jockey_id      0
trainer_id     0
owner_id       0
rank           0
umaban         0
wakuban        0
tansyo_odds    0
popularity     0
kinryou        0
sex            0
age            0
weight         0
weight_diff    0
dtype: int64

In [5]:
# 馬の過去成績テーブルの前処理
horse_results_preprocessd = preprocessing.process_horse_results()

In [6]:
horse_results_preprocessd

,horse_id,date,rank,prize,rank_diff,weather,race_type,course_len,ground_state,race_class,n_horses,time,win,rentai,show,place
0,2021101429,2024-11-09,9.0,0.0,0.2,0.0,1,1400,0.0,4.0,14.0,81.8,0,0,0,5
1,2021101429,2024-10-06,1.0,1140.0,0.0,1.0,1,1400,0.0,3.0,13.0,79.7,1,1,1,5
2,2021101429,2024-09-01,2.0,625.2,0.4,0.0,1,1400,0.0,5.0,7.0,82.0,0,1,1,4
3,2021101429,2024-08-10,2.0,629.2,0.0,0.0,1,1400,0.0,5.0,14.0,80.3,0,1,1,4
4,2021101429,2024-04-06,13.0,0.0,0.9,1.0,1,1600,1.0,7.0,16.0,95.3,0,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711582,2019100398,2022-01-27,2.0,52.0,0.4,1.0,0,1400,0.0,NaN,6.0,93.6,0,1,1,NaN
711583,2019100398,2022-01-04,2.0,48.0,0.0,0.0,0,1400,0.0,NaN,9.0,92.4,0,1,1,NaN
711584,2019100398,2021-12-16,1.0,70.0,0.0,0.0,0,1400,0.0,1.0,9.0,91.2,1,1,1,NaN
711585,2019100398,2021-10-26,2.0,14.0,0.1,1.0,0,1000,1.0,1.0,12.0,62.5,0,1,1,NaN


In [10]:
# 形式、欠損値を確認
horse_results_preprocessd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 700274 entries, 0 to 711586
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   horse_id          700274 non-null  int64         
 1   date              700274 non-null  datetime64[ns]
 2   rank              700274 non-null  float64       
 3   prize             700274 non-null  float64       
 4   rank_diff         699553 non-null  float64       
 5   weather           699658 non-null  float64       
 6   race_type         700274 non-null  int64         
 7   course_len        700274 non-null  int64         
 8   ground_condition  700269 non-null  float64       
 9   race_class        337222 non-null  float64       
 10  n_horses          700266 non-null  float64       
dtypes: datetime64[ns](1), float64(7), int64(3)
memory usage: 64.1 MB


In [83]:
# 欠損チェック
horse_results_preprocessd.isnull().sum()

horse_id                0
date                    0
rank                    0
prize                   0
rank_diff             260
weather               219
race_type               0
course_len              0
ground_condition        4
race_class          60543
n_horses                0
dtype: int64

In [16]:
%autoreload

In [17]:
# レース情報テーブルの加工
race_info_preprocess  = preprocessing.process_race_info()

In [12]:
race_info_preprocess

,race_id,date,race_type,around,course_len,weather,ground_state,race_class,place
0,201801010101,2018-07-28,1,0.0,1800,0,0.0,1.0,1
1,201801010102,2018-07-28,0,0.0,1000,0,0.0,1.0,1
2,201801010103,2018-07-28,0,0.0,2400,0,0.0,1.0,1
3,201801010104,2018-07-28,0,0.0,1700,0,0.0,1.0,1
4,201801010105,2018-07-28,1,0.0,1500,0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...
20725,202310030808,2023-09-03,0,0.0,1000,0,0.0,2.0,10
20726,202310030809,2023-09-03,1,0.0,1200,0,0.0,3.0,10
20727,202310030810,2023-09-03,0,0.0,1700,0,0.0,4.0,10
20728,202310030811,2023-09-03,1,0.0,1200,0,0.0,6.0,10


In [18]:
# 欠損チェック
race_info_preprocess.isnull().sum()

race_id                0
date                   0
race_type              0
around               897
course_len           161
weather                0
ground_condition    4787
race_class          1886
place                  0
dtype: int64

# 特徴量作成

In [10]:
from feature_producing import FeatureCreator
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
%autoreload

In [88]:
fp = FeatureCreator()
features = fp.create_features()

In [ ]:
"""
実行しようとしたところ、以下のようにメモリエラーが発生。５年分のデータを扱うため、メモリエラーが発生してしまった模様。
画面がフリーズ+落ちたので再起動して実行。アプリケーション立ち上げ直後実行したところ実行中にフリーズ。アプリを立ち上げ直して、データ加工を行って（cpu準備運動みたいな意味合い）、
ギリギリクリアした模様。

MemoryError: Unable to allocate 32.1 MiB for an array with shape (6, 700274) and data type int64

 """

'_\n実行しようとしたところ、以下のようにメモリエラーが発生。５年分のデータを扱うため、メモリエラーが発生してしまった模様。\n画面がフリーズ+落ちたので再起動して実行。アプリケーション立ち上げ直後実行したところ実行中にフリーズ。アプリを立ち上げ直して、データ加工を行って（cpu準備運動みたいな意味合い）、\nMemoryError: Unable to allocate 32.1 MiB for an array with shape (6, 700274) and data type int64\n\n    \n '

In [17]:
fp = FeatureCreator(
    poplation_filename="population_5year.csv",
    results_filename="results_5year.csv",
    horse_results_filename="horse_results.csv",
    race_info_filename="race_info_5year.csv"
    )
features = fp.create_features()

agg_horse_n_races_relative:   0%|          | 0/5 [00:00<?, ?it/s]

running agg_jockey()...
running agg_trainer()...


agg_horse_per_course_len:   0%|          | 0/6 [00:00<?, ?it/s]

agg_horse_per_ground_state_race_type:   0%|          | 0/6 [00:00<?, ?it/s]

agg_horse_per_race_class:   0%|          | 0/6 [00:00<?, ?it/s]

agg_horse_per_race_type:   0%|          | 0/6 [00:00<?, ?it/s]

running agg_sire()...


ValueError: You are trying to merge on object and int64 columns for key 'sire_id'. If you wish to proceed you should use pd.concat

In [ ]:
features

In [12]:
features

,race_id,date,horse_id,jockey_id,trainer_id,owner_id,rank,umaban,wakuban,tansyo_odds,...,trainer_n_races_ordinal_relative,trainer_winrate_ordinal_relative,trainer_n_races_turf_relative,trainer_winrate_turf_relative,trainer_n_races_dirt_relative,trainer_winrate_dirt_relative,trainer_winrate_relative,trainer_placerate_relative,trainer_showrate_relative,trainer_prize_relative
0,201801010101,2018-07-28,2016104880,5339,386,386,1,1,1,1.1,...,-1.685974,1.668507,1.255496,1.731266,-1.585492,1.306694,1.621231,1.772086,1.498110,1.936148
1,201801010101,2018-07-28,2016101840,5203,1148,1148,2,6,6,7.6,...,1.111034,0.186565,1.009612,0.206937,0.238420,0.505658,0.289002,0.035207,0.202538,-0.023312
2,201801010101,2018-07-28,2016105057,1127,1132,1132,3,3,3,8.3,...,0.085464,-0.025555,-0.242158,-0.079810,0.202657,0.526896,0.119830,0.199506,0.272840,-0.081080
3,201801010101,2018-07-28,2016103985,1085,434,434,4,4,4,46.7,...,-0.007769,-1.238270,-0.555101,-1.076889,0.274183,-1.508637,-1.296984,-1.138360,-1.494762,-0.796767
4,201801010101,2018-07-28,2016103821,1170,417,417,5,5,5,39.8,...,0.877950,-0.781692,-1.426869,-0.884949,1.418598,-0.678267,-0.768322,-0.704140,-0.621004,-0.709556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284763,202310030812,2023-09-03,2020105644,1037,1092,1092,8,1,1,62.8,...,0.637982,0.642620,0.093351,0.553046,0.817894,0.778641,0.693810,0.861246,0.856285,0.207988
284764,202310030812,2023-09-03,2018106584,1195,438,438,9,7,6,125.6,...,0.188843,0.056851,0.513430,1.071392,0.634686,0.482384,0.754361,0.372431,0.010234,1.672715
284765,202310030812,2023-09-03,2020101781,1018,1142,1142,10,2,2,9.8,...,-0.668605,-0.998387,-0.140026,-0.764008,-1.223570,-1.377632,-1.062160,-1.041642,-1.136027,-0.965433
284766,202310030812,2023-09-03,2019106647,1200,1032,1032,11,4,4,37.2,...,-0.893174,-0.906636,-1.291355,-1.405712,-0.281356,-0.567674,-0.941059,-0.954354,-0.985922,-1.125241


In [13]:
# 重複チェック
features.duplicated(subset=["race_id", "horse_id"]).sum()

np.int64(0)

In [14]:
#欠損チェック
features.isnull().sum()

race_id                             0
date                                0
horse_id                            0
jockey_id                           0
trainer_id                          0
                                 ... 
trainer_winrate_dirt_relative    5974
trainer_winrate_relative         5794
trainer_placerate_relative       5794
trainer_showrate_relative        5794
trainer_prize_relative           5794
Length: 539, dtype: int64

# 学習

In [ ]:
from training import Trainer
%load_ext autoreload

In [97]:
%autoreload

In [6]:
trainer = Trainer()
%load_ext autoreload

KeyboardInterrupt: 

In [8]:
# 距離、コースタイプの特徴を追加して実行
trainer = Trainer(config_filepath="config.yaml")
evaluation_df = trainer.run(
    valid_start_date="2022-10-01",
    test_start_date="2023-01-01",
    importance_filename="importance_course.csv",
    evaluation_filename="evaluation_course.csv"
)

Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.211196
[200]	valid_0's binary_logloss: 0.200536
[300]	valid_0's binary_logloss: 0.19569
[400]	valid_0's binary_logloss: 0.192285
[500]	valid_0's binary_logloss: 0.189766
[600]	valid_0's binary_logloss: 0.187562
[700]	valid_0's binary_logloss: 0.185168
[800]	valid_0's binary_logloss: 0.183198
[900]	valid_0's binary_logloss: 0.181424
[1000]	valid_0's binary_logloss: 0.179665
[1100]	valid_0's binary_logloss: 0.177897
[1200]	valid_0's binary_logloss: 0.176143
[1300]	valid_0's binary_logloss: 0.174519
[1400]	valid_0's binary_logloss: 0.172872
[1500]	valid_0's binary_logloss: 0.171364
[1600]	valid_0's binary_logloss: 0.169895
[1700]	valid_0's binary_logloss: 0.168251
[1800]	valid_0's binary_logloss: 0.166865
[1900]	valid_0's binary_logloss: 0.165372
[2000]	valid_0's binary_logloss: 0.163966
[2100]	valid_0's binary_logloss: 0.162608
[2200]	valid_0's binary_logloss: 0.161217
[2300]	valid_0's binary_

# 精度評価

In [1]:
from evaluationing import Evaluator
%load_ext autoreload

In [ ]:
%autoreload

In [2]:
evaluator = Evaluator()

In [ ]:
evaluator = Evaluator(evaluation_filename="evaluation_5year.csv")
